# 0. 개요 😀

모델은 기본적으로 ARIMA 에 피쳐 엔지니어링을 통해 생성한 값 diff (open - VWAP)를 학습 데이터로 사용했습니다.


- VWAP 및 피처 엔지니어링을 통해 생성한 값 diff 에 대해서는 아래 코드를 통해 설명드리겠습니다.



<br>

이번 시즌2 대회에서 집중했던 부분은 **안전**이었습니다. 🚧

<br>

이유는 크게 2가지입니다.

첫 번째로, 시즌1 대회에서 리더보드가 엄청 큰 범위로 변동되는 것을 겪었었습니다. 🤦🏻

<br>

두 번째로, 아무리 수익을 많이 거두어도 나중에 잘못된 타이밍에 매도를 하게 되면, 복리에 하락이 겹쳐 큰 손해를 입을 수 있다는 점입니다.

<br>

그렇기 때문에 트레이딩에 사용되는 여러 보조지표들 (VWAP, RSI)을 통해 하락이 예상되는 지점은 피하는 전략을 선택했습니다.


# 1. 개발 환경 🖥
개발 환경은 구글 colab을 사용했습니다. 

처음에는 낯설었지만, 다음과 같은 장점이 있었습니다.

<br>

1) 소음

맥을 사용한다면, 소음이 많이 생기는 경우가 있는데, colab을 사용하면 소음 없는 쾌적한 환경에서 개발이 가능합니다.

<br>

2) 웹 접근성

브라우저만 있으면 어디서든 개발할 수 있는 부분이 정말 좋았습니다.

# 2. 패키지 설치 및 임포트 🛠

In [1]:
# ARIMA 모델을 사용하기 위해 statsmodels 제일 최신 버전을 설치합니다.
!pip install statsmodels==0.12.2


In [2]:
# 1. 기본
# 데이터를 다루기 위한 pandas와 numpy를 import 해주었습니다.
import pandas as pd
import numpy as np


# 2. 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# 3. 유틸
# tqdm 패키지는 반복문에 대해 얼마나 진척되었는지를 가시적으로 확인할 수 있도록 도와줍니다.
# https://github.com/tqdm/tqdm 사용법은 정말 간단합니다.
from tqdm.auto import tqdm


# 4. 설정
# 경고가 나와서, 출력이 많아지지 않기 위해 ignore를 설정해주었습니다.
import warnings
warnings.filterwarnings('ignore')


# 5. stats models
# 시계열 모델을 위한 ARIMA를 임포트 해주었습니다.
from statsmodels.tsa.arima_model import ARIMA


# 6. 구글 드라이브
# colab이 구글 드라이와 연결되어 있기 때문에 임포트해주었습니다.
from google.colab import drive


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. 상수 🔐

In [5]:
# 1) 구글 드라이브 경로, 2) 데이터 경로, 3) 전체 경로를 상수로 정해주고 사용했습니다.
GOOGLE_DRIVE_PATH = "/content/drive"
DATA_PATH = "/content/drive/MyDrive/dacon /bit-traider"
SUBMIT_PATH = "/content/drive/MyDrive/dacon /bit-traider"


# 4. 데이터 로드 📥

In [6]:
 drive.mount(GOOGLE_DRIVE_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_x = pd.read_csv(DATA_PATH  + "/train_x_df.csv")
train_y = pd.read_csv(DATA_PATH  + "/train_y_df.csv")
test_x = pd.read_csv(DATA_PATH  + "/test_x_df.csv")


In [8]:
# train_x, train_y 를 sample_id 기준 하나로 합쳐서, train_z를 만들어줍니다.
# 사실 train x, y를 합치지 않아도 괜찮습니다만, 연속되는 변수 vwap, rsi, 등등을 만들기 위해 x, y를 합치고, 변수를 생성하고, x, y를 다시 분리하는 작업을 진행했습니다.
train_x["is_x"] = 1
train_y["is_x"] = 0
train_x_y = [train_x, train_y]
train_list = [x.set_index('sample_id') for x in train_x_y]

train_z = pd.concat(train_list, axis=0).rename_axis('sample_id').reset_index()


In [9]:
train_z.head(10)

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,is_x
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,1
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,1
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,1
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,1
5,0,5,0,0.995250,0.995286,0.994525,0.995033,448.190735,1229.981445,3.988398,89.858597,246.600739,1
6,0,6,0,0.995033,1.000435,0.994888,0.999637,13101.356445,36064.796875,69.978249,7477.168457,20581.027344,1
7,0,7,0,1.000508,1.002103,0.999275,1.002103,7941.424805,21915.634766,43.872372,5768.633301,15920.210938,1
8,0,8,0,1.001668,1.001813,0.999492,0.999928,4608.567871,12715.851562,28.281364,1374.416260,3792.563721,1
9,0,9,0,0.999928,0.999964,0.997680,0.997933,11251.487305,30983.783203,38.433647,3935.583740,10835.684570,1


# 5. 함수 작성 ✍️

### 1) VWAP
[거래량 가중 평균가(VWAP, Volume Weighted Average Price)](https://academy.binance.com/ko/articles/volume-weighted-average-price-vwap-explained)는 거래량에 가중치가 부여되어 특정 기간 동안의 평균 가격을 의미합니다.

<br>

트레이딩에서 VWAP은 상승세와 하락세를 판단하는 지표로 사용됩니다. **가격(open)이 vwap보다 크면 상승세**로 판단하고, 만약 **가격이 VWAP 보다 작으면 하락세**로 판단합니다.

![](https://raw.githubusercontent.com/skyepodium/bit-traider-image/master/image/vwap_formula.png)

<br>

저는 여기에 생각을 더해서... 

<br>

만약, 가격에서 VWAP을 뺀 값인 diff (open - VWAP)을 학습에 사용한다면, 2가지 이점이 있다고 생각했습니다.

<br>

첫 번째로 안전한 지점을 찾을 수 있는 것입니다. open - vwap이 양수이면, 상승 구간에 있기 때문에 안전한 지점을 찾을 수 있고, 적어도 손해는 안 볼 확률이 높아진다고 생각했습니다. 제일 가격이 높은 지점을 찾는 것이 아닌 안전한 지점을 찾는 문제로 바꾸었습니다.

<br>

두 번째로는 2개의 변수를 한 번에 사용할 수 있다는 점입니다. 개인적으로 여러 강의와 코드 샘플들을 보았지만, 변수가 저렇게 많은데 단 1가지 feature만 학습에 사용하는 것이 아쉬웠습니다. 그렇기 때문에  적어도 2개 이상의 변수를 함께 사용하면 조금 더 좋은 예측을 할것이라 생각했습니다.

In [10]:
# vwap과, open에서 vwap을 뺀 값인 diff를 계산합니다.
def make_vwap_and_diff(df):

    # 1) VAWP 계산

    # 일반적인 VWAP 공식에서 volume을 그대로 사용하지만, 여러번의 시도를 통해 tb_base_av 와 volume을 더했을 때 가장 좋은 volume이 나온다고 판단하여서 사용하였습니다.
    df["volume_tb_base_av"] = df["tb_base_av"] + df["volume"]
  #____________________________________________________________________________________________  
    # open하나만을 사용하기 보다는 open(시가), high(고가), low(저가) 3개의 평균을 price로 사용하였습니다. ------------------------------>이부분을 아래와 같이 수정
    #df['volume_price'] = ((df['open'] + df['high'] + df['low']) / 3) * df['volume_tb_base_av']
 
    # (open + close)/2 = price 로 사용할 것임.  그이유는 high와 low를 포함하는 값은 오차의 범위가 너무 커짐
    df['volume_price'] = ((df['open'] + df['close']) / 2) * df['volume_tb_base_av']
  #  ------------------------------------------------------------------------------------------

    # price와 volume의 곱의 합을 구해줍니다.
    df['volume_price_sum'] = df.groupby(['sample_id'])['volume_price'].apply(lambda x: x.cumsum())

    # volume의 합을 구해줍니다.
    df['volume_sum'] = df.groupby(['sample_id'])['volume_tb_base_av'].apply(lambda x: x.cumsum())

    # 2 변수의 나눗셈을 통해 vwap을 계산해줍니다.
    df['vwap'] = df['volume_price_sum'] / df['volume_sum']


    # 2) diff 계산
    # 매도수익이 open을 통해 이루어진다고 알려져있기 때문에 open에서 vwap을 뺀 값을 diff로 사용했습니다.          
    df["diff"] = df["open"] - df["vwap"]


    return df
    

In [11]:
# 데이터 프레임에서 sample_id 에 따른 open을 반환하는 함수입니다.
def get_open(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['open'].values
    

In [12]:
# 데이터 프레임에서 sample_id 에 따른 VWAP을 반환하는 함수입니다.
def get_vwap(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['vwap'].values
    

In [13]:
# 데이터 프레임에서 sample_id 에 따른 diff 반환하는 함수입니다.
def get_diff(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['diff'].values
    

In [14]:
# 데이터 프레임에서 sample_id 에 따른 rsi 반환하는 함수입니다.
def get_rsi(df,sample_id):
    
    return df[df["sample_id"] == sample_id]['rsi'].values
    

In [15]:
# 데이터 프레임에서 sample_id 에 따른 col_name 값을 반환하는 함수입니다.
def get_series(df,sample_id, col_name):
    
    return df[df["sample_id"] == sample_id][col_name].values
    

### 2) RSI
상대강도지수(relative strength index) 는 가격의 상승압력과 하락압력 간의 상대적인 강도를 나타냅니다.

<br>

트레이딩에서 사용되며, RSI 값이 30보다 작으면 초과매도로 판단하고, RSI 값이, 70 보다 크면 초과매수 상태로 판단합니다.

<br>

![](https://raw.githubusercontent.com/skyepodium/bit-traider-image/master/image/rsi_formula.png)

<br>

RSI는 최고점과, 최하점을 찾기 쉽다는 장점이 있습니다. 다만, 일명 박스권으로 천정과 바닥이 제대로 형성되지 않은 시장(RSI가 50근처 유지)에서는 유용하지 못합니다.

코인 데이터의 RSI 그래프를 그려보면 RSI 0 ~ 100 사이를 급변동 합니다.

<br>
저는 조금 더 안정적으로 투자하기 위해 65 초과인 상태를 초과매수국면으로 판단하고, 해당 시점 이후 50분동안은 투자하지 않도록 정해주었습니다.




In [16]:
# RSI를 만들어줍니다.
 #  ------------------------------------------------------------------------------------------
# 상승분, 하락분의 평균은 일반적으로 14일을 기준으로 생성합니다.----------------->수정
#상승분, 하락분의 평균을 14일 기준으로 하지 않고, 25일기준으로 더욱 더 안정성을 추구함
def make_rsi(df, period=25):
#  ------------------------------------------------------------------------------------------

    # 전일 대비 상승분을 계산해줍니다. - 상승분이 0보다 크면 상승분을 넣고, 0보다 작거나 같으면 0을 넣어줍니다.
    df["U"] = np.where(df.groupby(["sample_id"])["open"].diff(1) > 0, df.groupby(["sample_id"])["open"].diff(1), 0)

    # 전일 대비 하락분을 계산해줍니다. - 하락분이 0보다 작으면 하락분 * -1을 넣고, 0보다 크거나 같으면 0을 넣어줍니다.
    df["D"] = np.where(df.groupby(["sample_id"])["open"].diff(1) < 0, df.groupby(["sample_id"])["open"].diff(1) *(-1), 0)


    # 전일 대비 상승분의 평균을 계산해줍니다.
    ud_df = pd.DataFrame()
    ud_df["sample_id"] = df["sample_id"]
    ud_df["U"] = df["U"]
    ud_df["D"] = df["D"]

    # 상승분의 25일 평균을 구해줍니다.
    df["AU"] = ud_df.groupby(["sample_id"])["U"].rolling( window=period, min_periods=period ).mean().reset_index()["U"]
    # 하락분의 25일 평균을 구해줍니다.
    df["AD"] = ud_df.groupby(["sample_id"])["D"].rolling( window=period, min_periods=period ).mean().reset_index()["D"]


    # AU / (AU + AD) 의 백분율을 RSI 로 계산해줍니다.
    RSI = df["AU"] / (df["AU"] + df["AD"]) * 100
    
    df["rsi"] = RSI
    
    return df
    

# 6. 전처리 🪄

전처리 단계에서는 5번에서 작성한 함수를 train, test 데이터에 적용하여 피쳐 생성을 진행합니다.

In [17]:
# 1. train, test의 sample_id 목록을 저장합니다.
TRAIN_SAMPLE_ID_LIST = train_x["sample_id"].unique().tolist()
TEST_SAMPLE_ID_LIST = test_x["sample_id"].unique().tolist()


In [18]:
# 2. VWAP, diff 를 만들어줍니다.
test_x = make_vwap_and_diff(test_x)
train_z = make_vwap_and_diff(train_z)


In [19]:
# 3. rsi 를 만들어줍니다.
test_x = make_rsi(test_x, 25)
train_z = make_rsi(train_z, 25)


In [20]:
# 4. train x와 y를 분리합니다.
train_x = train_z[train_z["is_x"] == 1]
train_y = train_z[train_z["is_x"] == 0]

split_drop_cols = ["is_x"]

train_x = train_x.drop(columns=split_drop_cols, axis=1)
train_y = train_y.drop(columns=split_drop_cols, axis=1)


In [21]:
train_x.head(10)

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,volume_tb_base_av,volume_price,volume_price_sum,volume_sum,vwap,diff,U,D,AU,AD,rsi
0,0,0,0,0.993147,0.993546,0.992857,0.992966,1379.478027,3778.584961,11.240029,329.655548,903.091614,1709.133575,1697.266369,1697.266369,1709.133575,0.993057,0.000091,0.000000,0.000000,NaN,NaN,NaN
1,0,1,0,0.993256,0.993546,0.992712,0.992712,3438.807373,9419.426758,11.602611,1363.999268,3737.512695,4802.806641,4769.110420,6466.376790,6511.940216,0.993003,0.000253,0.000109,0.000000,NaN,NaN,NaN
2,0,2,0,0.992748,0.994815,0.992458,0.994815,3714.949463,10173.972656,19.579407,1222.802856,3350.688721,4937.752319,4907.048118,11373.424908,11449.692535,0.993339,-0.000591,0.000000,0.000508,NaN,NaN,NaN
3,0,3,0,0.994779,0.995286,0.994090,0.994996,2430.264648,6666.315430,15.591008,520.159546,1426.920776,2950.424194,2935.340505,14308.765414,14400.116730,0.993656,0.001123,0.002030,0.000000,NaN,NaN,NaN
4,0,4,0,0.994561,0.994779,0.993727,0.994779,3062.139404,8395.172852,15.228427,2166.334473,5939.279785,5228.473877,5200.606442,19509.371856,19628.590607,0.993926,0.000635,0.000000,0.000217,NaN,NaN,NaN
5,0,5,0,0.995250,0.995286,0.994525,0.995033,448.190735,1229.981445,3.988398,89.858597,246.600739,538.049332,535.435174,20044.807030,20166.639938,0.993959,0.001292,0.000689,0.000000,NaN,NaN,NaN
6,0,6,0,0.995033,1.000435,0.994888,0.999637,13101.356445,36064.796875,69.978249,7477.168457,20581.027344,20578.524902,20523.683478,40568.490507,40745.164841,0.995664,-0.000631,0.000000,0.000218,NaN,NaN,NaN
7,0,7,0,1.000508,1.002103,0.999275,1.002103,7941.424805,21915.634766,43.872372,5768.633301,15920.210938,13710.058105,13727.953599,54296.444107,54455.222946,0.997084,0.003423,0.005475,0.000000,NaN,NaN,NaN
8,0,8,0,1.001668,1.001813,0.999492,0.999928,4608.567871,12715.851562,28.281364,1374.416260,3792.563721,5982.984131,5987.756870,60284.200976,60438.207077,0.997452,0.004216,0.001160,0.000000,NaN,NaN,NaN
9,0,9,0,0.999928,0.999964,0.997680,0.997933,11251.487305,30983.783203,38.433647,3935.583740,10835.684570,15187.071045,15170.827778,75455.028754,75625.278122,0.997749,0.002179,0.000000,0.001740,NaN,NaN,NaN


### 1) 모델
모델은 diff (open - vwap)을 ARIMA를 통해 학습하여 생성했습니다. 

<br>

### 2) 제약조건
vwap, rsi의 마지막 값을 제약조건의 기준으로 사용했습니다.
vwap이 1보다 크면 open보다 vwap이 크다는 의미로 하향세에 접어들었다고 판단 투자하지 않았습니다.
(x의 open 의 마지막 값은 1입니다.)

<br>

rsi의 값이 65보다 크면 초과 매수 상태라고 판단하여 투자하지 않았습니다. 기본적으로 70 초과로 판단하는데 임의로 바꿀수 있는 값으로 조금 낮춰서 65로 시도했을때가 제일 좋아서 사용했습니다.

<br>

### 3) ARIMA와 p d q

개인적으로 ARIMA 모델의 order(p, d, q)에 투자를 많이 했었습니다.

ARIMA 모델의 AIC (아카이케 정보 기준) 가 가장 나오는 order를 brute force로 찾아보았지만, 

2시간 30분 ~ 3시간이 걸림에도 불구하고, 점수가 오히려 낮아져서, 여러번의 시도로 4, 0, 1 이 제일 좋다고 판단했습니다.

pac, acf 분석을 통해 AR 2 모델이라고 판단했습니다. 분석에는 다음 영상을 참고했습니다. (https://www.youtube.com/watch?v=-vSzKfqcTDg&t=360s)


In [22]:
result = []

for sample_id in tqdm(TEST_SAMPLE_ID_LIST):

    # 1. 데이터 로드
    # 1) diff - 학습에 사용
    diff_x = get_diff(test_x, sample_id)

    # 2) vwap - 보조 지표로 사용
    vwap_series = get_vwap(test_x, sample_id)

    # 3) rsi - 보조 지표로 사용
    rsi_series = get_rsi(test_x, sample_id)



    # 2. ARIMA
    # 1) 모델 정의
    ARIMA_MODEL = {}
    ARIMA_MODEL_FIT = {}

    # 2) AR 모델 적용
    try:
      ARIMA_MODEL = ARIMA(diff_x, order = (4,0,1))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 3) 수렴하지 않을 경우 p d q 를 1, 1, 0으로 사용
    except:
      ARIMA_MODEL = ARIMA(diff_x, order = (1,1,0))
      ARIMA_MODEL_FIT = ARIMA_MODEL.fit(trend = 'nc', full_output = True, disp = True)

    # 4) ARIMA 예측
    ARIMA_FORECAST  = ARIMA_MODEL_FIT.predict(1,120, typ='levels')

  #  ------------------------------------------------------------------------------------------
  # 매도 조건에 볼린저 제약 추가할 것임 -->민교 
  # 예를들면 np.argmax(ARIMA_FORECAST) 가 볼밴 상단이탈시 매도 
  #  ------------------------------------------------------------------------------------------

    # 3. 데이처 처리
    # 1) 최대 부분인 인덱스를 찾는데 해당 시점에 매도를 진행합니다.
    sell_time = np.argmax(ARIMA_FORECAST)
    
    # 2) 최대값을 찾습니다.
    max_val = np.max(ARIMA_FORECAST)
    
    # 3) vwap의 마지막 값을 가져옵니다.
    vwap_last_val = vwap_series[1379]

    rsi_last_val = rsi_series[1379]



    # 4. 투자 전략
    buy_quantity = 0

    # 1) 최대값이 0 보다 크면 가격이 vwap 보다 크다는 의미로, 투자합니다.
    if  max_val > 0:
        buy_quantity = 1


    # 2) 만약 vwap 마지막 값이, 1보다 크면 가격이 1보다 작다는 의미로 하향세이기 때문에 투자하지 않습니다.
    if vwap_last_val > 1 and sell_time < 50:
        buy_quantity = 0

    # 3) 만약 rsi의 값이 65 보다 크면, 초과매수 상태로 판단하여 투자하지 않습니다.
    if rsi_last_val > 65 and sell_time < 50:
        buy_quantity = 0



    # 5. 결과
    result_list = [
                   sample_id,
                   buy_quantity,
                   sell_time
                  ]

    result.append(result_list)
    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


볼린저밴드 추가 (민교)

볼린저 이평

볼린저 상단 = 20일 주가 이평 + 2*20일 주가 표준편차

볼린저 하단 = 20일 주가 이평 + 2*20일 주가 표준편차

In [ ]:
# bol_upper = ma + 2 * arr 여기서부터 수정해야함

# 8. 제출 🎉

In [24]:
# 1. 학습 결과를 데이터 프레임으로 만듭니다.

submit_columns = [
                  "sample_id", 
                  "buy_quantity", 
                  "sell_time"
                  ]


submit = pd.DataFrame(data=result, columns=submit_columns)


In [25]:
# 2. 결과 데이터 프레임 확인

submit.head(10)


,sample_id,buy_quantity,sell_time
0,7929,1,117
1,7930,1,48
2,7931,1,31
3,7932,0,22
4,7933,1,19
5,7934,1,116
6,7935,1,91
7,7936,1,119
8,7937,1,119
9,7938,1,61


In [26]:
# 3. 투자 개수 확인

submit[submit["buy_quantity"] == 1].shape[0]


617

In [27]:
# 4. sell_time 50미만에서 구매하는 개수 확인
cond1 = (submit["buy_quantity"] == 1)
cond2 = (submit["sell_time"] < 50)

submit[cond1 & cond2].shape[0]


134

In [ ]:
# 5. 제출

In [28]:
# 파일의 이름을 지정해줍니다.
FILE_NAME = "/0704_ARIMA_DIFF_VWAP_RSI_65_UNDER_50_SUBMIT.csv"


In [29]:
# 제출경로에 파일을 생성해줍니다.
RESULT_PATH = SUBMIT_PATH + FILE_NAME

submit.to_csv(RESULT_PATH, index=False)


# 9. 안정적 모델인지 어떻게 확인? 🤔
시즌 1을 경험하고, 최대한 안정적인 모델을 생성하기 위해 노력했습니다. 

다만, 내가 안정적이라고 생각해도, 객관적인 지표가 없다보니 얼마나 균형잡혀있는지 확인하기가 어려웠습니다.

이를 위해 public score를 다음과 같이 활용했습니다.

<br>

예를 들어, 
- open에 vwap을 섞어 투자개수가 줄었음에도 점수가 올라가는 현상
- vwap, rsi등 보조지표를 통해 sell_time 10, 20.. 50 미만은 투자하지 않기로 결정했음에도 점점 점수가 올라가는 현상
- rsi 초과매수로 상태를 70 초과가 아닌, 65 초과로, 더 안정적으로 결정해도 점수가 상승하는 현상
<br>

이렇게, 제약사항을 통해 보수적으로 투자했음에도 불구하고, 점수가 올라가는 경우에 집중하여, 해당 피쳐가 안정적으로 작용함을 판단했습니다.


# 10. 여러 아이디어 🥴
65번이라는 조금 많은 제출을 통해 여러 시도를 진행했습니다. 저의 개인적인 결과는 좋지 않았습니다만, 혹시나 이후 대회에서 다른 분들에게 조금이라도 도움이 될 수 있을까 해서 남기게 되었습니다.

<br>

### 1) RANDOM BOX 분류 모델
시계열 대회이지만, classification으로 생각해보았습니다.

결국에 제가 선택하는것은 120개중 1개이기 때문에 120개의 상자 중 1개를 선택했을때 그 값이 1보다 클 확률을 제가 얻을 수 있는 기대값이라고 생각했습니다. 

개인적으로 가장 창의적이라고 생각했지만, 잘 안되었습니다. 더 지니어스의 콩픈패스에 영감을 받았습니다.


<br>

### 2) 혼합 데이터
open에 vwap을 섞은것에 더하여, RSI, 이동평균선등을 모두 섞어 보았습니다. 생각보다 점수가 정말 좋았지만, vwap 한개를 섞은것보다는 아쉽다고 생각하여 사용하지는 않았습니다.

<br>



# 11. 다른 모델 사용여부 😁
ARIMA 이외에도 prophet, neural prophet, LSTM등 다양한 모델을 사용해보았습니다. 

다만, 더 좋은 결과를 주지 않았다고 판단했습니다.

또한, 위 모델들의 결과를 bagging하는 방법을 사용해보았지만, ARIMA 단 하나만을 사용했을때보다 예측 성능이 낮아진다고 판단하여 사용하지 않게 되었습니다.

